# Simulations of _E. coli_

This notebook contains simple tests of the E. coli model `iML1515` both in regular and ecModel format.

Benjamín J. Sánchez, 2019-10-15

## 1. Loading models

* Metabolic model: https://github.com/SysBioChalmers/ecModels/blob/chore/updateiML1515/eciML1515/model/eciML1515.xml
* ecModel: https://github.com/SysBioChalmers/ecModels/blob/chore/updateiML1515/eciML1515/model/eciML1515.xml

(temporal, eventually they will be available in the master branch)

In [1]:
import cobra
import os
import wget

# Metabolic model:
wget.download("https://raw.githubusercontent.com/SysBioChalmers/ecModels/chore/updateiML1515/eciML1515/model/iML1515.xml", "iML1515.xml", bar=False)
model = cobra.io.read_sbml_model("iML1515.xml")
os.remove("iML1515.xml")

# Enzyme-constrained model:
os.remove("eciML1515.xml")
wget.download("https://raw.githubusercontent.com/SysBioChalmers/ecModels/chore/updateiML1515/eciML1515/model/eciML1515.xml", "eciML1515.xml", bar=False)
ecModel = cobra.io.read_sbml_model("eciML1515.xml")
os.remove("eciML1515.xml")

## 2. Simulating models

### 2.1. Simulating the metabolic model

In [2]:
model.objective.expression.args[0]

1.0*BIOMASS_Ec_iML1515_core_75p37M

In [3]:
model.solver.timeout = 30
solution = model.optimize()
solution

,fluxes,reduced_costs
ALATA_D2,0.000000,-7.523353e-03
SHCHD2,0.000196,1.110223e-16
CPPPGO,0.000196,-2.168404e-19
GTHOr,0.217041,-5.561957e-17
DHORD5,0.000000,0.000000e+00
...,...,...
SUCCt1pp,0.000000,0.000000e+00
QUINDH,0.000000,-1.880838e-03
LCARSyi,0.000000,-1.880838e-03
BIOMASS_Ec_iML1515_core_75p37M,0.876997,-1.452221e-16


In [4]:
model.summary()

In [5]:
for reaction in model.reactions:
    if len(reaction.metabolites) == 1 and solution.fluxes[reaction.id] < 0:
        print(reaction.id + ": " + str(solution.fluxes[reaction.id]))

EX_pi_e: -0.8459567750195813
EX_mn2_e: -0.0006060050751690353
EX_fe2_e: -0.01408545226091154
EX_glc__D_e: -10.0
EX_zn2_e: -0.000299056050119596
EX_mg2_e: -0.0076079508351539465
EX_ca2_e: -0.004564770501092374
EX_ni2_e: -0.0002832701002599107
EX_cu2_e: -0.000621791025028721
EX_cobalt2_e: -2.1924930360674225e-05
EX_nh4_e: -9.47149537104813
EX_mobd_e: -6.138980500988783e-06
EX_so4_e: -0.220845438536999
EX_k_e: -0.17118371727564335
EX_o2_e: -22.13176323894588
EX_cl_e: -0.004564770501092374


Note that in total 16 metabolites are consumed: oxygen, glucose, ammonia, phosphate, sulphate and 11 minerals.

### 2.2. Simulating the enzyme-constrained model _as-is_

In [6]:
ecModel.objective.expression.args[0]

1.0*BIOMASS_Ec_iML1515_core_75p37M

In [7]:
ecModel.solver.timeout = 30
ecModel.optimize()

OptimizationError: solver status is 'unbounded'

### 2.3 Fixing the enzyme-constrained model

cobrapy cannot handle upper bounds = `Inf`, therefore we need to replace them with `1000` (standard in the field):

In [8]:
import math

for reaction in ecModel.reactions:
    if math.isinf(reaction.upper_bound):
        reaction.upper_bound = 1000

ecModel.optimize()

,fluxes,reduced_costs
EX_acgam_e,0.0,0.0
EX_cellb_e,0.0,0.0
EX_chol_e,0.0,0.0
EX_pi_e,1000.0,0.0
EX_h_e,1000.0,0.0
...,...,...
prot_Q59385_exchange,0.0,-0.0
prot_Q6BEX0_exchange,0.0,-0.0
prot_Q6BF16_exchange,0.0,-0.0
prot_Q6BF17_exchange,0.0,-0.0


Pretty high objective function -> we need to lower the upper bound of glucose uptake:

In [9]:
ecModel.reactions.EX_glc__D_e_REV.upper_bound = 10
ecModel.optimize()

,fluxes,reduced_costs
EX_acgam_e,0.0,0.0
EX_cellb_e,0.0,0.0
EX_chol_e,0.0,0.0
EX_pi_e,1000.0,0.0
EX_h_e,1000.0,0.0
...,...,...
prot_Q59385_exchange,0.0,-0.0
prot_Q6BEX0_exchange,0.0,-0.0
prot_Q6BF16_exchange,0.0,-0.0
prot_Q6BF17_exchange,0.0,-0.0


Nothing changed -> let's look at the summary of in/out fluxes:

In [10]:
ecModel.summary()

We see that there are many uptake fluxes fully unconstrained. Let's fix all of them to zero except for the original 16:

In [11]:
for reaction in ecModel.reactions:
    if len(reaction.metabolites) == 1 and reaction.name.endswith(" (reversible)"):
        reaction.lower_bound = 0
        reaction.upper_bound = 0

ecModel.reactions.EX_glc__D_e_REV.upper_bound = 10   #glucose will be limiting
ecModel.reactions.EX_pi_e_REV.upper_bound = 1000
ecModel.reactions.EX_mn2_e_REV.upper_bound = 1000
ecModel.reactions.EX_fe2_e_REV.upper_bound = 1000
ecModel.reactions.EX_zn2_e_REV.upper_bound = 1000
ecModel.reactions.EX_mg2_e_REV.upper_bound = 1000
ecModel.reactions.EX_ca2_e_REV.upper_bound = 1000
ecModel.reactions.EX_ni2_e_REV.upper_bound = 1000
ecModel.reactions.EX_cu2_e_REV.upper_bound = 1000
ecModel.reactions.EX_cobalt2_e_REV.upper_bound = 1000
ecModel.reactions.EX_nh4_e_REV.upper_bound = 1000
ecModel.reactions.EX_mobd_e_REV.upper_bound = 1000
ecModel.reactions.EX_so4_e_REV.upper_bound = 1000
ecModel.reactions.EX_k_e_REV.upper_bound = 1000
ecModel.reactions.EX_o2_e_REV.upper_bound = 1000
ecModel.reactions.EX_cl_e_REV.upper_bound = 1000

ecModel.optimize()

,fluxes,reduced_costs
EX_acgam_e,0.000000,-0.281185
EX_cellb_e,0.000000,-0.361121
EX_chol_e,0.000000,-0.022570
EX_pi_e,999.154043,0.000000
EX_h_e,0.000000,0.000000
...,...,...
prot_Q59385_exchange,0.000000,-0.000000
prot_Q6BEX0_exchange,0.000000,-0.000000
prot_Q6BF16_exchange,0.000000,-0.000000
prot_Q6BF17_exchange,0.000000,-0.000000


Success!!

## 3. Correcting model fields:

In [12]:
# Metabolite ids: remove the trailing "[comp]" (if any):
for metabolite in ecModel.metabolites:
    trail = "[" + metabolite.compartment + "]"
    if metabolite.id.endswith(trail):
        metabolite.id = metabolite.id.split(trail)[0]    

## 4. Model export

In [13]:
#Save model as .xml & .json:
cobra.io.write_sbml_model(ecModel, "eciML1515.xml")
cobra.io.save_json_model(ecModel, "eciML1515.json")

## 5. Changing GAM

In [14]:
print(model.reactions.get_by_id('BIOMASS_Ec_iML1515_core_75p37M'))

BIOMASS_Ec_iML1515_core_75p37M: 0.000223 10fthf_c + 2.6e-05 2fe2s_c + 0.000223 2ohph_c + 0.00026 4fe4s_c + 0.513689 ala__L_c + 0.000223 amet_c + 0.295792 arg__L_c + 0.241055 asn__L_c + 0.241055 asp__L_c + 75.55223 atp_c + 2e-06 btn_c + 0.005205 ca2_c + 0.005205 cl_c + 0.000576 coa_c + 2.5e-05 cobalt2_c + 0.133508 ctp_c + 0.000709 cu2_c + 0.09158 cys__L_c + 0.026166 datp_c + 0.027017 dctp_c + 0.027017 dgtp_c + 0.026166 dttp_c + 0.000223 fad_c + 0.006715 fe2_c + 0.007808 fe3_c + 0.26316 gln__L_c + 0.26316 glu__L_c + 0.612638 gly_c + 0.215096 gtp_c + 70.028756 h2o_c + 0.094738 his__L_c + 0.290529 ile__L_c + 0.195193 k_c + 0.019456 kdo2lipid4_e + 0.450531 leu__L_c + 0.343161 lys__L_c + 0.153686 met__L_c + 0.008675 mg2_c + 0.000223 mlthf_c + 0.000691 mn2_c + 7e-06 mobd_c + 0.013894 murein5px4p_p + 0.001831 nad_c + 0.000447 nadp_c + 0.013013 nh4_c + 0.000323 ni2_c + 0.063814 pe160_p + 0.075214 pe161_p + 0.185265 phe__L_c + 0.000223 pheme_c + 0.221055 pro__L_c + 0.000223 pydx5p_c + 0.000223 r

In [15]:
model.reactions.get_by_id('BIOMASS_Ec_iML1515_core_75p37M').add_metabolites({model.metabolites.atp_c: -10.0})
model.reactions.get_by_id('BIOMASS_Ec_iML1515_core_75p37M').add_metabolites({model.metabolites.h2o_c: -10.0})
model.reactions.get_by_id('BIOMASS_Ec_iML1515_core_75p37M').add_metabolites({model.metabolites.adp_c: -10.0})
model.reactions.get_by_id('BIOMASS_Ec_iML1515_core_75p37M').add_metabolites({model.metabolites.h_c: -10.0})
model.reactions.get_by_id('BIOMASS_Ec_iML1515_core_75p37M').add_metabolites({model.metabolites.pi_c: -10.0})

In [16]:
print(model.reactions.get_by_id('BIOMASS_Ec_iML1515_core_75p37M'))

BIOMASS_Ec_iML1515_core_75p37M: 0.000223 10fthf_c + 2.6e-05 2fe2s_c + 0.000223 2ohph_c + 0.00026 4fe4s_c + 0.513689 ala__L_c + 0.000223 amet_c + 0.295792 arg__L_c + 0.241055 asn__L_c + 0.241055 asp__L_c + 85.55223 atp_c + 2e-06 btn_c + 0.005205 ca2_c + 0.005205 cl_c + 0.000576 coa_c + 2.5e-05 cobalt2_c + 0.133508 ctp_c + 0.000709 cu2_c + 0.09158 cys__L_c + 0.026166 datp_c + 0.027017 dctp_c + 0.027017 dgtp_c + 0.026166 dttp_c + 0.000223 fad_c + 0.006715 fe2_c + 0.007808 fe3_c + 0.26316 gln__L_c + 0.26316 glu__L_c + 0.612638 gly_c + 0.215096 gtp_c + 80.028756 h2o_c + 0.094738 his__L_c + 0.290529 ile__L_c + 0.195193 k_c + 0.019456 kdo2lipid4_e + 0.450531 leu__L_c + 0.343161 lys__L_c + 0.153686 met__L_c + 0.008675 mg2_c + 0.000223 mlthf_c + 0.000691 mn2_c + 7e-06 mobd_c + 0.013894 murein5px4p_p + 0.001831 nad_c + 0.000447 nadp_c + 0.013013 nh4_c + 0.000323 ni2_c + 0.063814 pe160_p + 0.075214 pe161_p + 0.185265 phe__L_c + 0.000223 pheme_c + 0.221055 pro__L_c + 0.000223 pydx5p_c + 0.000223 r